# Linkage and Blocking Evaluation and Tuning Tool

The purpose of this tool is to look at the various options for implementing blocking in [anonlink](https://anonlink-entity-service.readthedocs.io/en/stable/) for CODI datasets. Currently, anonlink uses blocklib library which supports two blocking methods:

- “p-sig”: Probabilistic signature
- “lambda-fold”: LSH based lambda-fold

which are proposed by the following publications:

- [Scalable Entity Resolution Using Probabilistic Signatures on Parallel Databases](https://arxiv.org/abs/1712.09691)
- [An LSH-Based Blocking Approach with a Homomorphic Matching Technique for Privacy-Preserving Record Linkage](https://www.computer.org/csdl/journal/tk/2015/04/06880802/13rRUxASubY)

Adjustments to the blocklib configuration will be made to the type of blocking, encoding, and threshold. We will evaluate multiple runs of our linkage tools using an example data set. The metrics for evaluation include:

- Precision
- Recall
- Reduction Ratio
- Set completeness
- Performance based on average block size

## Useful Terminology

- Blocking - a technique that makes record linkage scalable. It is achieved by partitioning datasets into groups, called blocks and only comparing records in corresponding blocks. This can reduce the number of comparisons that need to be conducted to find which pairs of records should be linked.
- Bloom filter - a probabilistic data structure used to test set membership. It tells if an element may be in a set, or definitely isn't.
- Precision – how many of the found matches are actual matches (found groups : true matches)
- Recall – how many of the actual matches we found (true matches : found groups)
- Reduction Ratio – measures the proportion of number of comparisons reduced by using blocking
- Set Completeness – how many true matches are maintained after blocking
- Feature hashing – a fast and space-efficient way of vectorizing features, i.e. turning arbitrary features into indices in a vector or matrix
- `p-sig` signature – A subrecord of an entity that can be used to uniquely link commonality between multiple records of an entity
- `p-sig` Blocking keys – lower the cost of comparison between datasets by selecting partitions of the raw records (ex. First name, last name, postal code) *its assumed records sharing no blocking keys do not match with each other*


## Setting up the Environment

- The basic requirement is that you have [data-owner-tools](https://github.com/mitre/data-owner-tools) set up with all of Synthetic Denver sites extracted via extract.py and named with the format pii_\*.csv (e.g. pii_ch.csv) for all 5 sites (scripts can easily be adjusted to work with other data) 

In [39]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
from IPython.display import FileLink, FileLinks
import qgrid
import csv
import json
from pathlib import Path
import dcctools.config
from itertools import combinations
import importlib
import time

### Setting up a new run:

Set an optional run identifier / description string for the row that will be created in the table of run data

In [66]:
run_description = 'No Blocking Run 1 (New Data Set)'

Set path to data-owner-tools project:

In [67]:
DATA_OWNER_TOOLS_DIR = '/Users/apellitieri/Desktop/CDC/CODI/data-owner-tools/'

Ensure the blocking schema file to use for the run is set correctly in `config.json`:

In [69]:
CONFIG = dcctools.config.Configuration("config.json")
print('Config file in use:')
with open('config.json', 'r') as config:
    config_data = json.load(config)
    print(json.dumps(config_data, indent=2))
blocking_schema_file = ''
if CONFIG.blocked():
    blocking_schema_file = CONFIG.blocking_schema()
    with open(blocking_schema_file, 'r') as blocking_schema:
        schema_data = json.load(blocking_schema)
        print('\nBlocking schema being used:')
        print(json.dumps(schema_data, indent=2))
else:
    blocking_schema_file = 'None'
    print('\nNo blocking set to be used on this run.')

Config file in use:
{
  "systems": [
    "site_a",
    "site_b",
    "site_c",
    "site_d",
    "site_e",
    "site_f"
  ],
  "projects": [
    "name-sex-dob-phone",
    "name-sex-dob-zip",
    "name-sex-dob-parents",
    "name-sex-dob-addr"
  ],
  "schema_folder": "/Users/apellitieri/Desktop/CDC/CODI/data-owner-tools/example-schema",
  "inbox_folder": "/Users/apellitieri/Desktop/CDC/CODI/inbox",
  "matching_results_folder": "/Users/apellitieri/Desktop/CDC/CODI/results",
  "output_folder": "/Users/apellitieri/Desktop/CDC/CODI/output",
  "entity_service_url": "http://localhost:8851/api/v1",
  "matching_threshold": 0.8,
  "mongo_uri": "localhost:27017",
  "blocked": false,
  "blocking_schema": "/Users/apellitieri/Desktop/CDC/CODI/data-owner-tools/example-blocking-schema/lambda.json"
}

No blocking set to be used on this run.


## Garble and block with anonlink

The following block runs the script to garble the pii_\*.csv files and then block and package the CLKs into the inbox folder.

[TODO]: Make the garble scripts not specific to the synthetic denver sites - currently need to change name of script depending on using synthetic denver or the new site

[TODO]: Figure out a way to record the blocking statistics from the run. The anonlink client blocking program prints out statistics about the blocking run but does not provide the output in an easily digestable format. The important output to look at is the maximum and average block size. From the [anonlink-client documentation](https://anonlink-client.readthedocs.io/en/stable/tutorial/Blocking%20with%20Anonlink%20Entity%20Service.html#Blocking):
```
The record linkage run time will be largely dominated by the maximum block size, and the number of blocks. In general the smaller the average block size, the better.
```

In [70]:
inbox_folder = CONFIG.inbox_folder()
garble_start = time.perf_counter()
if CONFIG.blocked():
    shell_script = "{}testing-and-tuning/blocking-garble.sh".format(DATA_OWNER_TOOLS_DIR)
    !$shell_script {blocking_schema_file} {inbox_folder}
else:
    shell_script = "{}testing-and-tuning/garble.sh".format(DATA_OWNER_TOOLS_DIR)
    !$shell_script {inbox_folder}
garble_end = time.perf_counter()
garble_time = garble_end - garble_start

Cleaning inbox...
Running garble.py for A
CLK data written to output/name-sex-dob-phone.json
CLK data written to output/name-sex-dob-zip.json
CLK data written to output/name-sex-dob-parents.json
CLK data written to output/name-sex-dob-addr.json
Running garble.py for B
CLK data written to output/name-sex-dob-phone.json
CLK data written to output/name-sex-dob-zip.json
CLK data written to output/name-sex-dob-parents.json
CLK data written to output/name-sex-dob-addr.json
Running garble.py for C
CLK data written to output/name-sex-dob-phone.json
CLK data written to output/name-sex-dob-zip.json
CLK data written to output/name-sex-dob-parents.json
CLK data written to output/name-sex-dob-addr.json
Running garble.py for D
CLK data written to output/name-sex-dob-phone.json
CLK data written to output/name-sex-dob-zip.json
CLK data written to output/name-sex-dob-parents.json
CLK data written to output/name-sex-dob-addr.json
Running garble.py for E
CLK data written to output/name-sex-dob-phone.json

In [71]:
print(f"Garble and block (if enabled) took {garble_time:0.2f} seconds")

Garble and block (if enabled) took 15.18 seconds


In [1]:
!python validate.py

All necessary input is present


In [2]:
# Need to drop database collection here if previous run took place
# database.match_groups.drop()

In [74]:
match_start = time.perf_counter()
!python match.py
match_end = time.perf_counter()
match_time = match_end - match_start

{'current_stage': {'description': 'waiting for CLKs', 'number': 1, 'progress': {'absolute': 6, 'description': 'number of parties already contributed', 'relative': 1.0}}, 'stages': 3, 'state': 'created', 'time_added': '2021-05-03T09:15:58.160066+00:00', 'time_started': None}
{'current_stage': {'description': 'waiting for CLKs', 'number': 1, 'progress': {'absolute': 6, 'description': 'number of parties already contributed', 'relative': 1.0}}, 'stages': 3, 'state': 'created', 'time_added': '2021-05-03T09:15:58.160066+00:00', 'time_started': None}
{'current_stage': {'description': 'waiting for CLKs', 'number': 1, 'progress': {'absolute': 6, 'description': 'number of parties already contributed', 'relative': 1.0}}, 'stages': 3, 'state': 'created', 'time_added': '2021-05-03T09:15:58.160066+00:00', 'time_started': None}
{'current_stage': {'description': 'waiting for CLKs', 'number': 1, 'progress': {'absolute': 6, 'description': 'number of parties already contributed', 'relative': 1.0}}, 'stag

Inserted 100 of 1386 records.
Inserted 200 of 1386 records.
Inserted 300 of 1386 records.
Inserted 400 of 1386 records.
Inserted 500 of 1386 records.
Inserted 600 of 1386 records.
Inserted 700 of 1386 records.
Inserted 800 of 1386 records.
Inserted 900 of 1386 records.
Inserted 1000 of 1386 records.
Inserted 1100 of 1386 records.
Inserted 1200 of 1386 records.
Inserted 1300 of 1386 records.
{'current_stage': {'description': 'waiting for CLKs', 'number': 1, 'progress': {'absolute': 6, 'description': 'number of parties already contributed', 'relative': 1.0}}, 'stages': 3, 'state': 'created', 'time_added': '2021-05-03T09:16:16.510600+00:00', 'time_started': None}
{'current_stage': {'description': 'waiting for CLKs', 'number': 1, 'progress': {'absolute': 6, 'description': 'number of parties already contributed', 'relative': 1.0}}, 'stages': 3, 'state': 'created', 'time_added': '2021-05-03T09:16:16.510600+00:00', 'time_started': None}
{'current_stage': {'description': 'waiting for CLKs', 'n

{'current_stage': {'description': 'compute output', 'number': 3}, 'stages': 3, 'state': 'running', 'time_added': '2021-05-03T09:16:16.510600+00:00', 'time_started': '2021-05-03T09:16:26.601474+00:00'}
{'current_stage': {'description': 'compute output', 'number': 3}, 'stages': 3, 'state': 'completed', 'time_added': '2021-05-03T09:16:16.510600+00:00', 'time_completed': '2021-05-03T09:16:34.731951+00:00', 'time_started': '2021-05-03T09:16:26.601474+00:00'}
{'groups': [[[0, 245], [5, 318]], [[1, 374], [2, 324]], [[0, 242], [5, 79]], [[0, 719], [2, 297]], [[0, 77], [1, 482], [5, 442]], [[0, 9], [2, 299]], [[1, 729], [4, 353]], [[0, 663], [3, 243]], [[1, 369], [2, 19]], [[2, 139], [3, 699]], [[2, 209], [3, 337], [1, 408], [4, 729]], [[1, 740], [5, 465], [0, 54]], [[0, 722], [4, 305]], [[0, 244], [2, 285]], [[4, 660], [5, 459]], [[4, 630], [5, 525]], [[4, 534], [5, 161]], [[4, 471], [5, 243]], [[4, 408], [5, 255]], [[4, 404], [5, 540]], [[4, 321], [5, 481], [3, 200]], [[4, 213], [5, 378]], [[

Inserted 100 of 1339 records.
Inserted 200 of 1339 records.
Inserted 300 of 1339 records.
Inserted 400 of 1339 records.
Inserted 500 of 1339 records.
Inserted 600 of 1339 records.
Inserted 700 of 1339 records.
Inserted 800 of 1339 records.
Inserted 900 of 1339 records.
Inserted 1000 of 1339 records.
Inserted 1100 of 1339 records.
Inserted 1200 of 1339 records.
Inserted 1300 of 1339 records.
{'current_stage': {'description': 'waiting for CLKs', 'number': 1, 'progress': {'absolute': 6, 'description': 'number of parties already contributed', 'relative': 1.0}}, 'stages': 3, 'state': 'created', 'time_added': '2021-05-03T09:16:40.651483+00:00', 'time_started': None}
{'current_stage': {'description': 'waiting for CLKs', 'number': 1, 'progress': {'absolute': 6, 'description': 'number of parties already contributed', 'relative': 1.0}}, 'stages': 3, 'state': 'created', 'time_added': '2021-05-03T09:16:40.651483+00:00', 'time_started': None}
{'current_stage': {'description': 'waiting for CLKs', 'n

{'current_stage': {'description': 'compute output', 'number': 3}, 'stages': 3, 'state': 'running', 'time_added': '2021-05-03T09:16:40.651483+00:00', 'time_started': '2021-05-03T09:16:50.728153+00:00'}
{'current_stage': {'description': 'compute output', 'number': 3}, 'stages': 3, 'state': 'running', 'time_added': '2021-05-03T09:16:40.651483+00:00', 'time_started': '2021-05-03T09:16:50.728153+00:00'}
{'current_stage': {'description': 'compute output', 'number': 3}, 'stages': 3, 'state': 'running', 'time_added': '2021-05-03T09:16:40.651483+00:00', 'time_started': '2021-05-03T09:16:50.728153+00:00'}
{'current_stage': {'description': 'compute output', 'number': 3}, 'stages': 3, 'state': 'completed', 'time_added': '2021-05-03T09:16:40.651483+00:00', 'time_completed': '2021-05-03T09:16:59.708352+00:00', 'time_started': '2021-05-03T09:16:50.728153+00:00'}
{'groups': [[[1, 374], [2, 324]], [[0, 242], [5, 79]], [[0, 245], [5, 318]], [[0, 330], [4, 418]], [[0, 77], [1, 482], [5, 442]], [[1, 729],

Inserted 100 of 1385 records.
Inserted 200 of 1385 records.
Inserted 300 of 1385 records.
Inserted 400 of 1385 records.
Inserted 500 of 1385 records.
Inserted 600 of 1385 records.
Inserted 700 of 1385 records.
Inserted 800 of 1385 records.
Inserted 900 of 1385 records.
Inserted 1000 of 1385 records.
Inserted 1100 of 1385 records.
Inserted 1200 of 1385 records.
Inserted 1300 of 1385 records.
{'current_stage': {'description': 'waiting for CLKs', 'number': 1, 'progress': {'absolute': 6, 'description': 'number of parties already contributed', 'relative': 1.0}}, 'stages': 3, 'state': 'created', 'time_added': '2021-05-03T09:17:05.855705+00:00', 'time_started': None}
{'current_stage': {'description': 'waiting for CLKs', 'number': 1, 'progress': {'absolute': 6, 'description': 'number of parties already contributed', 'relative': 1.0}}, 'stages': 3, 'state': 'created', 'time_added': '2021-05-03T09:17:05.855705+00:00', 'time_started': None}
{'current_stage': {'description': 'waiting for CLKs', 'n

Inserted 100 of 1466 records.
Inserted 200 of 1466 records.
Inserted 300 of 1466 records.
Inserted 400 of 1466 records.
Inserted 500 of 1466 records.
Inserted 600 of 1466 records.
Inserted 700 of 1466 records.
Inserted 800 of 1466 records.
Inserted 900 of 1466 records.
Inserted 1000 of 1466 records.
Inserted 1100 of 1466 records.
Inserted 1200 of 1466 records.
Inserted 1300 of 1466 records.
Inserted 1400 of 1466 records.


In [75]:
print(f"Matching took {match_time:0.2f} seconds")

Matching took 89.88 seconds


In [76]:
!python link_ids.py

results/link_ids.csv created


In [77]:
!python patid_translate.py

results/patid_link_ids.csv created


## Record Results

In [79]:
RESULTS_PATH = '/Users/apellitieri/Desktop/CDC/CODI/results'
ANSWER_KEY_CSV = '/Users/apellitieri/Desktop/CDC/CODI/new_answer_key.csv'

In [80]:
run_precision = 0
run_recall = 0
run_f_score = 0
answer_key_length = 0
proposed_pairs_count = 0

systems = CONFIG.systems()

true_positives = 0
false_positives = 0

answer_key = []

with open(ANSWER_KEY_CSV) as ak_csv:
  ak_reader = csv.reader(ak_csv)
  next(ak_reader)
  for row in ak_reader:
    if row[3] == '1':
      answer_pair = [row[1], row[2]]
      answer_pair.sort()
      answer_key.append(answer_pair)

answer_key_length = len(answer_key)

patid_csv_path = Path(RESULTS_PATH) / "patid_link_ids.csv"

with open(patid_csv_path) as patid_csv:
  pat_id_reader = csv.reader(patid_csv)
  next(pat_id_reader)
  for row in pat_id_reader:
    patids = row[1:6]
    patids = list(filter(lambda id: len(id) > 0, patids))
    combos = combinations(patids, 2)
    for a, b in combos:
      pair = [a, b]
      pair.sort()
      if pair in answer_key:
        true_positives += 1
      else:
        false_positives += 1

run_precision = true_positives / (true_positives + false_positives)
run_recall = true_positives / answer_key_length
run_f_score = 2 * ((run_precision * run_recall) / (run_precision + run_recall))
proposed_pairs_count = true_positives + false_positives

In [81]:
print(f"Precision: {run_precision:0.2f}\nRecall: {run_recall:0.2f}\nF-Score: {run_f_score:0.2f}")

Precision: 1.00
Recall: 0.56
F-Score: 0.72


In [82]:
with open('tuning-files-scripts/example_run_data.csv', 'a', newline='') as csvfile:
    fieldnames = ['Run Description', 'Blocking', 'Precision',
                  'Recall', 'F-Score', 'Answer Key Size',
                  'Proposed Pairs', 'Garble & Block Time (s)',
                  'Match Time (s)', 'Blocking Config']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    # writer.writeheader() # Remove after initial run
    writer.writerow({
        'Run Description': run_description, 'Blocking': CONFIG.blocked(),
        'Precision': run_precision, 'Recall': run_recall, 'F-Score': run_f_score,
        'Answer Key Size': answer_key_length, 'Proposed Pairs': proposed_pairs_count,
        'Garble & Block Time (s)': garble_time,
        'Match Time (s)': match_time, 'Blocking Config': blocking_schema_file
    })
print("Successfully added run to example_run_data.csv!")

Successfully added run to example-run-data.csv!


In [2]:
pd.read_csv('tuning-files-scripts/example_run_data.csv')

NameError: name 'pd' is not defined